In [ ]:
import os
import pandas as pd

pd.options.plotting.backend = "plotly"

In [ ]:
with open('default.env', 'r') as file:
    var_name = 'path_dir_COVID19_repo'
    first_line = file.readline().strip()
    assert var_name + '=' == first_line[:len(var_name + '=')]
    path_dir_COVID19_repo = first_line[len(var_name + '='):]

In [ ]:
def prepare_df_states(df):
    return df.drop(columns=['Province/State', 'Lat', 'Long']).groupby('Country/Region').sum()

In [ ]:
# Extracting data from csv files and creating dataframes
# Defining paths to files
path_dir_data = os.path.join(path_dir_COVID19_repo, 'csse_covid_19_data', 'csse_covid_19_time_series')
path_confirmed = os.path.join(path_dir_data, 'time_series_covid19_confirmed_global.csv')
path_deaths = os.path.join(path_dir_data, 'time_series_covid19_deaths_global.csv')
path_recovered = os.path.join(path_dir_data, 'time_series_covid19_recovered_global.csv')

# Creation of dataframes
df_confirmed = prepare_df_states(pd.read_csv(path_confirmed))
df_deaths = prepare_df_states(pd.read_csv(path_deaths))
df_recovered = prepare_df_states(pd.read_csv(path_recovered))

In [ ]:
df_france_deaths = df_deaths.loc[['France'], :].T
df_france_deaths.plot(title='Cumulated covid deaths in France')

In [ ]:
df_france_deaths_daily = df_france_deaths.diff()
df_france_deaths_daily.columns = ['daily']
df_france_deaths_weekly_average = df_france_deaths_daily.rolling(7).mean()
df_france_deaths_daily.loc[:, 'weekly average'] = df_france_deaths_weekly_average
df_france_deaths_daily.plot(title='Number of daily covid deaths in France')

In [ ]:
# Manually setting start & ends of covid tides (for the sake of quickness)
index_tide1_start = df_france_deaths_daily.index.tolist().index(('3/4/20'))
index_tide1_stop = df_france_deaths_daily.index.tolist().index(('6/30/20'))
index_tide2_start = df_france_deaths_daily.index.tolist().index(('10/5/20'))
index_tide2_stop = None
df_len = df_france_deaths_daily.shape[0]

In [ ]:
# tide 1
df_tide1 = df_france_deaths_daily.iloc[index_tide1_start:index_tide1_stop, :].reset_index()
df_tide1.columns = ['tide1, date', 'tide1, daily', 'tide1, weekly average']

# tide 2
df_tide2 = df_france_deaths_daily.iloc[index_tide2_start:index_tide2_stop, :].reset_index()
df_tide2.columns = ['tide2, date', 'tide2, daily', 'tide2, weekly average']

# merging data of the 2 tides
df_tides_compare = df_tide1.merge(df_tide2, how='outer', left_index=True, right_index=True)

In [ ]:
df_tides_compare[['tide1, daily', 'tide2, daily']].\
    plot(title='Number of daily covid deaths in France, raw data')

In [ ]:
df_tides_compare[['tide1, weekly average', 'tide2, weekly average']].\
    plot(title='Number of daily covid deaths in France, weekly average')